# Librerías y funciones

In [67]:
import itertools
import pandas as pd
import re
import networkx as nx
from pyvis.network import Network
from IPython.display import display, HTML

import magaz_analysis as ma

In [68]:
# Escala de prueba en formato str dummy
test_scale = "101010101010" # Hexátona
test_scale = "101011010101" # Diatónica
test_scale

'101011010101'

In [69]:
# Pasa a lista de números
ma.do_numbers(test_scale)

[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]

In [70]:
# Pasa a lista de letras
ma.do_strings(ma.do_numbers(test_scale))

'101011010101'

In [71]:
# Multiplica dos listas
ma.lxl(test_scale, test_scale)

[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]

In [72]:
# Calcula vector interválico
ma.do_vector(test_scale)

'254361'

In [73]:
# Calcula perfil interválico
ma.do_profile(test_scale)

'2212221'

In [74]:
# Calcula una rotación de la escala para centrarla en Do
ma.do_rootmode("000011010101")

'110101010000'

In [75]:
# Calcula anillo 1
ma.do_ring1(test_scale)

{'scale': '101011010101',
 'analysis': [[0, 1],
  [2, 1],
  [2, 3],
  [4, 3],
  [5, 6],
  [7, 6],
  [7, 8],
  [9, 8],
  [9, 10],
  [11, 10]],
 'ring': ['110110101010',
  '110011010101',
  '100111010101',
  '101101010101',
  '101010110101',
  '101011100101',
  '101011001101',
  '101011011001',
  '101011010011',
  '101011010110']}

# Generación de datos

In [76]:
# Genero tabla 
df = ma.generate_data()

print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12287 entries, 0 to 2046
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   scale      12287 non-null  object
 1   length     12287 non-null  int64 
 2   profile    12287 non-null  object
 3   vector     12287 non-null  object
 4   dim_ring1  12287 non-null  int64 
 5   ring1      12286 non-null  object
dtypes: int64(2), object(4)
memory usage: 671.9+ KB
None


,scale,length,profile,vector,dim_ring1,ring1
0,100000000001,2,111,100000,2,100000000010
0,100000000001,2,111,100000,2,100000000010
1,100000000010,2,102,010000,4,110000000000
1,100000000010,2,102,010000,4,100000000100
1,100000000010,2,102,010000,4,100000000100


# Exploración básica

In [77]:
# Saco heptatónicas
df[df["length"]==7].head()

,scale,length,profile,vector,dim_ring1,ring1
62,100000111111,7,6111111,654321,2,100001111110
62,100000111111,7,6111111,654321,2,100001011111
94,100001011111,7,5211111,554331,4,100010111110
94,100001011111,7,5211111,554331,4,100010011111
94,100001011111,7,5211111,554331,4,100000111111


In [78]:
# Saco escalas con un vector interválico concreto
df[df["vector"]=="654321"].head()

,scale,length,profile,vector,dim_ring1,ring1
62,100000111111,7,6111111,654321,2,100001111110
62,100000111111,7,6111111,654321,2,100001011111
1054,110000011111,7,1611111,654321,2,101000011111
1054,110000011111,7,1611111,654321,2,110000101111
1550,111000001111,7,1161111,654321,2,110100001111


In [79]:
# Saco info de la escala de prueba
df[df["scale"]==test_scale]

,scale,length,profile,vector,dim_ring1,ring1
724,101011010101,7,2212221,254361,10,110110101010
724,101011010101,7,2212221,254361,10,110011010101
724,101011010101,7,2212221,254361,10,100111010101
724,101011010101,7,2212221,254361,10,101101010101
724,101011010101,7,2212221,254361,10,101010110101
724,101011010101,7,2212221,254361,10,101011100101
724,101011010101,7,2212221,254361,10,101011001101
724,101011010101,7,2212221,254361,10,101011011001
724,101011010101,7,2212221,254361,10,101011010011
724,101011010101,7,2212221,254361,10,101011010110


In [80]:
# Saco info de todas las escalas en anillo 1 de la escala de prueba (diatónica)
ma.df_find_ring1(df, test_scale)


,scale,length,profile,vector,dim_ring1,ring1
0,100111010101,7,3112221,343542,8,101110101010
1,100111010101,7,3112221,343542,8,101011010101
2,100111010101,7,3112221,343542,8,100110110101
3,100111010101,7,3112221,343542,8,100111100101
4,100111010101,7,3112221,343542,8,100111001101
...,...,...,...,...,...,...
83,110110101010,7,1212222,254442,10,110110011010
84,110110101010,7,1212222,254442,10,110110110010
85,110110101010,7,1212222,254442,10,110110100110
86,110110101010,7,1212222,254442,10,110110101100


# Estudio de heptatónicas

In [100]:
# Saco todas las heptatónicas
hepta = df[df["length"]==7]

# Filtro heptatónicas útiles
filtro1 = hepta.apply(lambda r: r.scale.count("000")==0, axis=1)
filtro2 = hepta.apply(lambda r: r.ring1.count("000")==0, axis=1)
filtro3 = hepta.apply(lambda r: r.scale.count("1111")==0, axis=1)
hepta = hepta[filtro1*filtro2*filtro3]

# Saco heptatónicas básicas con etiquetas
hepta_bas = pd.DataFrame([(i,k) for i in v for k,v in ma.generate_hepta_bas(c=False).items()],columns=["scale","source"])

# Etiqueto heptatónicas
hepta = pd.merge(hepta,hepta_bas,how="left",on="scale")

# Saco info
print(hepta.info())
hepta.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1414 entries, 0 to 1413
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   scale      1414 non-null   object
 1   length     1414 non-null   int64 
 2   profile    1414 non-null   object
 3   vector     1414 non-null   object
 4   dim_ring1  1414 non-null   int64 
 5   ring1      1414 non-null   object
 6   source     0 non-null      object
dtypes: int64(2), object(5)
memory usage: 88.4+ KB
None


,scale,length,profile,vector,dim_ring1,ring1,source
0,100100110111,7,3312111,435432,6,101001101110,NaN
1,100100110111,7,3312111,435432,6,100101010111,NaN
2,100100110111,7,3312111,435432,6,100100101111,NaN
3,100100110111,7,3312111,435432,6,100100111011,NaN
4,100100111011,7,3311211,434541,6,101001110110,NaN


In [135]:
# Saco heptatónicas válidas aún no agrupadas
hepta_out = hepta[hepta["source"].isna()]["scale"].unique()

# Saco sus agrupaciones
d = {"scale":[],"source":[]}
for h1 in hepta_out:
    v = []
    for i,h2 in enumerate(hepta_out):
        if ma.is_mode(h1,h2):
            v.append(str(i))
    d["scale"].append(h1)
    d["source"].append("".join(v))

# Cuento la dimensionalidad de los grupos
df_anl = pd.DataFrame(d)
df_anl["size"] = df_anl.groupby("source")["scale"].transform("size")

# Me quedo con los grupos de 7
df_anl = df_anl[df_anl["size"]==7]

df_anl

,scale,source,size
1,100100111011,13294109171178210,7
3,100101011011,3639295112172181,7
4,100101011101,46870105113159206,7
6,100101101011,6648696115156183,7
7,100101101101,7718791116160169,7
...,...,...,...
206,111011001010,46870105113159206,7
207,111011001100,143073121132161207,7
208,111011010010,113578106136164208,7
209,111011010100,31424581144166209,7


In [65]:
# Para actualizar cambios a magaz_analysis.py

import importlib
importlib.reload(ma)

<module 'magaz_analysis' from '/Users/hector/Desktop/music_analysis/magaz_analysis.py'>